In [116]:
"""
install ImageNet label and store into label
"""
import os
os.system("wget -O ImageNet_label https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt")

label = []
with open("ImageNet_label") as f:
    label = [i.split("'")[1] for i in f.readlines()]

print(label)

--2021-08-17 13:35:19--  https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30564 (30K) [text/plain]
Saving to: ‘ImageNet_label’

     0K .......... .......... .........                       100% 3.27M=0.009s

2021-08-17 13:35:19 (3.27 MB/s) - ‘ImageNet_label’ saved [30564/30564]



['tench, Tinca tinca', 'goldfish, Carassius auratus', 'great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias', 'tiger shark, Galeocerdo cuvieri', 'hammerhead, hammerhead shark', 'electric ray, crampfish, numbfish, torpedo', 'stingray', 'cock', 'hen', 'ostrich, Struthio camelus', 'brambling, Fringilla montifringilla', 'goldfinch, Carduelis carduelis', 'house finch, linnet, Carpodacus mexicanus', 'junco, snowbird', 'indigo bunting, indigo finch, indigo bird, Passerina cyanea', 'robin, American robin, Turdus migratorius', 'bulbul', 'jay', 'magpie', 'chickadee', 'water ouzel, dipper', 'kite', 'bald eagle, American eagle, Haliaeetus leucocephalus', 'vulture', 'great grey owl, great gray owl, Strix nebulosa', 'European fire salamander, Salamandra salamandra', 'common newt, Triturus vulgaris', 'eft', 'spotted salamander, Ambystoma maculatum', 'axolotl, mud puppy, Ambystoma mexicanum', 'bullfrog, Rana catesbeiana', 'tree frog, tree-frog', 'tailed frog, bell toad, 

In [117]:
import time
class timer():
    def __init__(self):
        self.startTime=0
        self.endTime=0
        self.totaltime=0
    def start(self):
        self.startTime=time.time()
    def end(self):
        self.totaltime =time.time()-self.startTime

In [118]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
		std=[0.229, 0.224, 0.225])
])

In [119]:
"""
save pytorch resnet model
"""
import torch
import torchvision

resnet50 = torchvision.models.resnet50(pretrained=True, progress=False).eval()
#print(resnet50)
torch.save(resnet50,"resnet.pt")

In [120]:
"""
load pytorch model
"""
load_torch_timer = timer()
load_torch_timer.start()
resnet_pytorch = torch.load("resnet.pt")
load_torch_timer.end()
print("load pytorch model: %f s" % (load_torch_timer.totaltime))



load pytorch model: 0.034955 s


In [121]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
img = Image.open('cat.png')
img = transform(img)
img = torch.unsqueeze(img, 0)
print(img.size())

torch.Size([1, 3, 224, 224])


In [122]:
""" pytorch inference """
infer_torch_timer = timer()
infer_torch_timer.start()
output = resnet_pytorch(img)
infer_torch_timer.end()

print(output.size())
output = output.squeeze()
print(output.size())
print("pytorch time: %fs"%(infer_torch_timer.totaltime))

sum = 0
for i in output:
    sum += i
# print(output)

torch.Size([1, 1000])
torch.Size([1000])
pytorch time: 0.044099s


In [123]:
"""
pytorch result
"""

import torch.nn.functional as F
_, index = torch.sort(output, descending=True)
percentage = F.softmax(output)

result_torch = [(label[i],percentage[i].item()*100) for i in index[:5]]
for j in result_torch:
    print("%-25s | %08.5f"%(j[0],j[1])+"%")

tabby, tabby cat          | 43.14913%
tiger cat                 | 29.68253%
Egyptian cat              | 25.96065%
tiger, Panthera tigris    | 00.34236%
lynx, catamount           | 00.30307%


In [124]:
"""
image process and transform
"""
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
img = Image.open('cat.png')
img = transform(img)
img = torch.unsqueeze(img, 0)
print(img.size())

torch.Size([1, 3, 224, 224])


In [125]:
import torch.onnx
BATCH_SIZE=1
dummy_input=torch.randn(BATCH_SIZE, 3, 224, 224)
torch.onnx.export(resnet50, dummy_input, "resnet.onnx", verbose=False,input_names = ['input'], output_names = ['output'])

In [126]:
"""
load onnx model
"""

import onnxruntime as rt
load_onnx_timer = timer()
load_onnx_timer.start()
resnet_onnx = rt.InferenceSession("resnet.onnx")
load_onnx_timer.end()
print("load pytorch model: %f s" % (load_onnx_timer.totaltime))

load pytorch model: 0.172642 s


In [127]:
""" 
onnx inference 
"""

def to_np(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

infer_onnx_timer = timer()
ort_input = {resnet_onnx.get_inputs()[0].name:to_np(img)}
infer_onnx_timer.start()
output = resnet_onnx.run(None,ort_input)[0][0]
infer_onnx_timer.end()

output = torch.tensor(output)

In [128]:
_, index = torch.sort(output, descending=True)
percentage = F.softmax(output)

print(index[:5])
result_onnx = [(label[i],percentage[i].item()*100) for i in index[:5]]
for j in result_onnx:
    print("%-25s | %08.5f"%(j[0],j[1])+"%")

tensor([281, 282, 285, 292, 287])
tabby, tabby cat          | 43.14905%
tiger cat                 | 29.68259%
Egyptian cat              | 25.96067%
tiger, Panthera tigris    | 00.34236%
lynx, catamount           | 00.30307%


In [129]:
print("load time")
print("Pytorch : %fs" % (load_torch_timer.totaltime))
print("onnx    : %fs" % (load_onnx_timer.totaltime))

print("-"*50)
print("infer time")
print("Pytorch : %fs" % (infer_torch_timer.totaltime))
print("onnx    : %fs" % (infer_onnx_timer.totaltime))

load time
Pytorch : 0.034955s
onnx    : 0.172642s
--------------------------------------------------
infer time
Pytorch : 0.044099s
onnx    : 0.098687s


In [130]:
import tensorrt as trt

print(trt.__version__)

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

def build_engine(onnx_path, shape = [1,224,224,3]):

   """
   create the TensorRT engine
   Args:
      onnx_path : Path to onnx_file. 
      shape : Shape of the input of the ONNX file. 
  """
   with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
       builder.max_workspace_size = (256 << 20)
       """256 MB"""
       with open(onnx_path, 'rb') as model:
           parser.parse(model.read())
       network.get_input(0).shape = shape
       engine = builder.build_cuda_engine(network)
       return engine

def save_engine(engine, file_name):
   buf = engine.serialize()
   with open(file_name, 'wb') as f:
       f.write(buf)

def load_engine(trt_runtime, plan_path):
   with open(plan_path, 'rb') as f:
       engine_data = f.read()
   engine = trt_runtime.deserialize_cuda_engine(engine_data)
   return engine

7.2.3.4


In [131]:
from onnx import ModelProto

BATCH_SIZE=1
dummy_input=torch.randn(BATCH_SIZE, 3, 224, 224)
resnet50 = torchvision.models.resnet50(pretrained=True, progress=False).eval()
torch.onnx.export(resnet50, dummy_input, "resnet.onnx", verbose=False,input_names = ['input'], output_names = ['output'])

onnx_path = "resnet.onnx"
trt_path  = "resnet.plan"

model = ModelProto()
with open(onnx_path, "rb") as f:
  model.ParseFromString(f.read())

batch_size = 1
d0 = model.graph.input[0].type.tensor_type.shape.dim[1].dim_value
d1 = model.graph.input[0].type.tensor_type.shape.dim[2].dim_value
d2 = model.graph.input[0].type.tensor_type.shape.dim[3].dim_value
shape = [batch_size , d0, d1 ,d2]

"""
build engine and start timer to get total build time
"""
build_trt_timer = timer()
build_trt_timer.start()
trt_engine = build_engine(onnx_path,shape)
build_trt_timer.end()


"""
save engine as trt_path
"""
save_engine(trt_engine,trt_path)

In [132]:
print("build a tensorRT engine: %f s" %(build_trt_timer.totaltime))

build a tensorRT engine: 8.557570 s


In [133]:
import pycuda.driver as cuda
import pycuda.autoinit 

def allocate_buffers(engine, batch_size, data_type):
   """
   This is the function to allocate buffers for input and output in the device
   args:
      engine      : The path to the TensorRT engine. 
      batch_size  : The batch size for execution time.
      data_type   : The type of the data for input and output, for example trt.float32. 
   
   return:
      h_input_1   : Input in the host.
      d_input_1   : Input in the device. 
      h_output_1  : Output in the host. 
      d_output_1  : Output in the device. 
      stream      : CUDA stream.
   """

   # Determine dimensions and create page-locked memory buffers (which won't be swapped to disk) to hold host inputs/outputs.
   h_input_1 = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(0)), dtype=trt.nptype(data_type))
   h_output = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(1)), dtype=trt.nptype(data_type))
   # Allocate device memory for inputs and outputs.
   d_input_1 = cuda.mem_alloc(h_input_1.nbytes)

   d_output = cuda.mem_alloc(h_output.nbytes)
   # Create a stream in which to copy inputs/outputs and run inference.
   stream = cuda.Stream()
   return h_input_1, d_input_1, h_output, d_output, stream 

def load_images_to_buffer(pics, pagelocked_buffer):
   preprocessed = np.asarray(pics).ravel()
   np.copyto(pagelocked_buffer, preprocessed) 

def do_inference(engine, pics_1, h_input_1, d_input_1, h_output, d_output, stream, batch_size):
   """
   This is the function to run the inference
   Args:
      engine      : Path to the TensorRT engine 
      pics_1      : Input images to the model.  
      h_input_1   : Input in the host         
      d_input_1   : Input in the device 
      h_output_1  : Output in the host 
      d_output_1  : Output in the device 
      stream      : CUDA stream
      batch_size  : Batch size for execution time
   
   Output:
      The list of output images

   """

   load_images_to_buffer(pics_1, h_input_1)

   with engine.create_execution_context() as context:

      """ copy data from host to device """
      cuda.memcpy_htod_async(d_input_1, h_input_1, stream)

      # Run inference.
      context.profiler = trt.Profiler()
      context.execute(batch_size=1, bindings=[int(d_input_1), int(d_output)])

      # Transfer predictions back from the GPU.
      cuda.memcpy_dtoh_async(h_output, d_output, stream)

      # Synchronize the stream
      stream.synchronize()
      
      # Return the host output.
      
      return h_output

In [134]:
"""
load tensorRT engine
"""

load_trt_timer = timer()
load_trt_timer.start()
resnet_trt = load_engine(trt_runtime,trt_path)
load_trt_timer.end()

In [135]:
print("load a tensorRT engine: %f s" %(load_trt_timer.totaltime))

load a tensorRT engine: 0.482223 s


In [136]:
"""
inference step
   workflow:
      1. allocate buffer for input and output in GPU
      2. copy data from host to device
      3. run inference in GPU
      4. copy result from GPU to the host
      5. reshape (optional)
"""

""" allocate buffer """

host_input, device_input, host_output, device_output, stream = allocate_buffers(
    engine = resnet_trt,
    batch_size = 1,
    data_type = trt.float32)

In [137]:
img = Image.open('cat.png')
img = transform(img)
img = torch.unsqueeze(img, 0)
print(img.size())

torch.Size([1, 3, 224, 224])


In [138]:
""" inference """
infer_trt_timer = timer()
infer_trt_timer.start()
output = do_inference(
    engine    = resnet_trt,
    pics_1    = img,
    h_input_1 = host_input,
    d_input_1 = device_input,
    h_output  = host_output,
    d_output  = device_output,
    stream    = stream,
    batch_size= 1,
)
infer_trt_timer.end()

Conv_0 + Relu_1: 0.080672ms
MaxPool_2: 0.017856ms
Conv_3 + Relu_4: 0.019488ms
Conv_5 + Relu_6: 0.050592ms
Conv_7: 0.04064ms
Conv_8 + Add_9 + Relu_10: 0.046656ms
Conv_11 + Relu_12: 0.043808ms
Conv_13 + Relu_14: 0.050176ms
Conv_15 + Add_16 + Relu_17: 0.046112ms
Conv_18 + Relu_19: 0.04416ms
Conv_20 + Relu_21: 0.05008ms
Conv_22 + Add_23 + Relu_24: 0.046144ms
Conv_22 + Add_23 + Relu_24 output reformatter 0: 0.03056ms
Conv_25 + Relu_26: 0.064032ms
Conv_27 + Relu_28: 0.0736ms
Conv_29: 0.036736ms
Conv_30 + Add_31 + Relu_32: 0.06864ms
Conv_33 + Relu_34: 0.038592ms
Conv_35 + Relu_36: 0.073408ms
Conv_37 + Add_38 + Relu_39: 0.042816ms
Conv_40 + Relu_41: 0.038336ms
Conv_42 + Relu_43: 0.07392ms
Conv_44 + Add_45 + Relu_46: 0.042944ms
Conv_47 + Relu_48: 0.038592ms
Conv_49 + Relu_50: 0.073504ms
Conv_51 + Add_52 + Relu_53: 0.04336ms
Conv_54 + Relu_55: 0.06176ms
Conv_56 + Relu_57: 0.133248ms
Conv_58: 0.06096ms
Conv_59 + Add_60 + Relu_61: 0.107104ms
Conv_59 + Add_60 + Relu_61 output reformatter 0: 0.01152

In [139]:
print(type(output))
""" transform ndarray to Tensor """
output = torch.tensor(output)
print(type(output))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [140]:
_, index = torch.sort(output, descending=True)
percentage = F.softmax(output)

print(index[:5])
result_trt = [(label[i],percentage[i].item()*100) for i in index[:5]]
for j in result_trt:
    print("%-25s | %08.5f"%(j[0],j[1])+"%")

tensor([281, 282, 285, 292, 287])
tabby, tabby cat          | 43.14908%
tiger cat                 | 29.68258%
Egyptian cat              | 25.96064%
tiger, Panthera tigris    | 00.34236%
lynx, catamount           | 00.30307%


In [141]:
print("load time")
print("Pytorch      : %fs" % (load_torch_timer.totaltime))
print("onnx runtime : %fs" % (load_onnx_timer.totaltime))
print("tensorRT     : %fs" % (load_trt_timer.totaltime))

print("-"*50)
print("infer time")
print("Pytorch      : %fs" % (infer_torch_timer.totaltime))
print("onnx runtime : %fs" % (infer_onnx_timer.totaltime))
print("tensorRT     : %fs" % (infer_trt_timer.totaltime))

print("-"*50)
print("result")
print("  Pytorch:")
for j in result_torch:
    print("     %-25s | %08.5f"%(j[0],j[1])+"%")

print(" "*5+"-"*45)
print("  onnx:")
for j in result_onnx:
    print("     %-25s | %08.5f"%(j[0],j[1])+"%")

print(" "*5+"-"*45)
print("  tensorRT:")
for j in result_trt:
    print("     %-25s | %08.5f"%(j[0],j[1])+"%")

load time
Pytorch      : 0.034955s
onnx runtime : 0.172642s
tensorRT     : 0.482223s
--------------------------------------------------
infer time
Pytorch      : 0.044099s
onnx runtime : 0.098687s
tensorRT     : 0.008694s
--------------------------------------------------
result
  Pytorch:
     tabby, tabby cat          | 43.14913%
     tiger cat                 | 29.68253%
     Egyptian cat              | 25.96065%
     tiger, Panthera tigris    | 00.34236%
     lynx, catamount           | 00.30307%
     ---------------------------------------------
  onnx:
     tabby, tabby cat          | 43.14905%
     tiger cat                 | 29.68259%
     Egyptian cat              | 25.96067%
     tiger, Panthera tigris    | 00.34236%
     lynx, catamount           | 00.30307%
     ---------------------------------------------
  tensorRT:
     tabby, tabby cat          | 43.14908%
     tiger cat                 | 29.68258%
     Egyptian cat              | 25.96064%
     tiger, Panthera tigris 



### reference and issues:

* issue: **AttributeError: 'tensorrt.tensorrt.Builder' object has no attribute 'max_workspace_size'** 
    * error when nvidia-tensorrt is 8.x.x.x version
    * sol: remove nvidia-tensorrt 8.x.x.x and install 7.2.
    * ref: [(github issue link)](https://github.com/NVIDIA-AI-IOT/torch2trt/issues/557)
* issue: **AttributeError: module 'google.protobuf.descriptor' has no attribute '_internal_create_key**
    * error when ```protobuf``` version is  not equal to ```protoc```
        * show ```protobuf``` version: ```pip3 show protobuf```
        * show ```protoc``` version: ```protoc --version```
    * sol: 
        1. update protobuf version:  ```pip3 install --upgrade protobuf``` 
        2. install ```protoc``` if not exist:
            1. ```wget https://github.com/protocolbuffers/protobuf/releases/download/v3.17.3/protoc-3.17.3-linux-x86_64.zip```
            2. ```unzip protoc-3.17.3-linux-x86_64.zip```
            3. ```vim ~/.bashprofile``` and paste ```PATH=$PATH:/home/YOUR_HOST_NAME/bin```
            4. ```source ~/.bashprofile``` and ```echo $PATH``` to check whether ```/home/YOUR_HOST_NAME/bin``` exist or not.
    * ref: [stackoverflow linke](https://stackoverflow.com/questions/61922334/how-to-solve-attributeerror-module-google-protobuf-descriptor-has-no-attribu)
* [Imagenet label](https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt)
* [NVIDIA onnx to tensorRT](https://developer.nvidia.com/blog/speeding-up-deep-learning-inference-using-tensorflow-onnx-and-tensorrt/)
    * [zh-cn (注意中文文檔中 ```save_engine``` 中的 code 有錯)](https://developer.nvidia.com/zh-cn/blog/speeding-up-deep-learning-inference-using-tensorflow-onnx-and-tensorrt/)
* [NVIDIA Deep Learning TensorRT Documentation](https://docs.nvidia.com/deeplearning/tensorrt/developer-guide/index.html#python_topics)
* [Jetson Nano 运用TensorRT加速引擎 上篇](https://www.rs-online.com/designspark/nvidia-cudagpujetson-nano-tensorrt-cn)
* [Jetson Nano 运用TensorRT加速引擎 下篇](https://www.rs-online.com/designspark/nvidia-cudagpujetson-nano-tensorrt-3-cn)